#### Testing combinations with starts and stops as indices

This is intended to be used with oamap version ( I know we don't intend to finally use it, but let's go with it for now).

##### Idea for calculation

Using the kernel structure used in test_combinations, we can  generate combinations arrays for every event. 

For that, each event is assigned a block ( i.e. each block in the grid serves as an event), and the combinations is calculated for each block using the `starts[block_index]` and event length.
This is appended to `left` and `right` arrays. 

In [1]:
import pycuda.autoinit
import pycuda
import pycuda.driver as cuda
import pycuda.gpuarray as gpuarray
from pycuda.compiler import *
import numpy

In [2]:
# The generation step. Let's form a random integer array, from which we will form starts 
# and stops arrays
base_len = 32
base_arr = numpy.random.randint(6, size=base_len)

In [3]:
cumul_arr = numpy.cumsum(base_arr)

In [4]:
start = cumul_arr[:-1]
stop = cumul_arr[1:]

In [5]:
start

array([ 5,  7, 10, 11, 13, 16, 21, 26, 26, 30, 31, 33, 37, 39, 39, 39, 44,
       49, 50, 55, 60, 65, 65, 70, 70, 70, 74, 79, 82, 83, 84])

In [6]:
stop[:5]

array([ 7, 10, 11, 13, 16])

In [7]:
lengths = stop-start
pairs_lengths = numpy.zeros(len(lengths)+1, dtype=numpy.int32)
pairs_lengths[1:] = numpy.cumsum(lengths*lengths)
lengths_arr = numpy.array([base_len]).astype(numpy.int32)
lengths = lengths.astype(numpy.int32)

In [8]:
pairs_lengths

array([  0,   4,  13,  14,  18,  27,  52,  77,  77,  93,  94,  98, 114,
       118, 118, 118, 143, 168, 169, 194, 219, 244, 244, 269, 269, 269,
       285, 310, 319, 320, 321, 346], dtype=int32)

In [9]:
lengths

array([2, 3, 1, 2, 3, 5, 5, 0, 4, 1, 2, 4, 2, 0, 0, 5, 5, 1, 5, 5, 5, 0, 5,
       0, 0, 4, 5, 3, 1, 1, 5], dtype=int32)

##### Error 

The left and right arrays aren't changing at all, they remain as zeroes. Can't figure out why this is happening.

In [10]:
# Now let's form the cuda function

mod = SourceModule('''
__global__ void comb_events(int* left,int* right,int* start,int* length,int* lengths,int* pairs_lengths)
{
    int i = blockIdx.x*blockDim.x + threadIdx.x;
    int j =  blockIdx.y*blockDim.y+threadIdx.y;
    int k =  blockIdx.z*blockDim.z+threadIdx.z;
    if (i <length[0] && j< lengths[i] && k<lengths[i])
    {
        left[pairs_lengths[i] + j*lengths[i] + k] = j + start[i];
        right[pairs_lengths[i] + j*lengths[i] + k] = k + start[i];
    }
}
''')

In [11]:
func = mod.get_function("comb_events")

In [12]:
left = numpy.zeros(pairs_lengths[-1]).astype(numpy.int32)
right = numpy.zeros(pairs_lengths[-1]).astype(numpy.int32)

In [13]:
func(cuda.InOut(left), cuda.InOut(right), cuda.In(start), cuda.In(lengths_arr),
    cuda.In(lengths), cuda.In(pairs_lengths), block=(1, 8, 8), grid=(base_len, 1))

In [14]:
left

array([ 5,  5,  6,  6,  0,  0,  0,  1,  1,  1,  2,  2,  2,  7,  0,  0,  1,
        1, 10, 10, 10, 11, 11, 11, 12, 12, 12,  0,  0,  0,  0,  0,  1,  1,
        1,  1,  1,  2,  2,  2,  2,  2,  3,  3,  3,  3,  3,  4,  4,  4,  4,
        4, 11, 11, 11, 11, 11, 12, 12, 12, 12, 12, 13, 13, 13, 13, 13, 14,
       14, 14, 14, 14, 15, 15, 15, 15, 15, 13, 13, 13, 13, 14, 14, 14, 14,
       15, 15, 15, 15, 16, 16, 16, 16,  0, 16, 16, 17, 17,  0,  0,  0,  0,
        1,  1,  1,  1,  2,  2,  2,  2,  3,  3,  3,  3, 21, 21, 22, 22,  0,
        0,  0,  0,  0,  1,  1,  1,  1,  1,  2,  2,  2,  2,  2,  3,  3,  3,
        3,  3,  4,  4,  4,  4,  4, 26, 26, 26, 26, 26, 27, 27, 27, 27, 27,
       28, 28, 28, 28, 28, 29, 29, 29, 29, 29, 30, 30, 30, 30, 30,  0, 30,
       30, 30, 30, 30, 31, 31, 31, 31, 31, 32, 32, 32, 32, 32, 33, 33, 33,
       33, 33, 34, 34, 34, 34, 34,  0,  0,  0,  0,  0,  1,  1,  1,  1,  1,
        2,  2,  2,  2,  2,  3,  3,  3,  3,  3,  4,  4,  4,  4,  4, 31, 31,
       31, 31, 31, 32, 32

In [15]:
right

array([ 5,  6,  5,  6,  0,  1,  2,  0,  1,  2,  0,  1,  2,  7,  0,  1,  0,
        1, 10, 11, 12, 10, 11, 12, 10, 11, 12,  0,  1,  2,  3,  4,  0,  1,
        2,  3,  4,  0,  1,  2,  3,  4,  0,  1,  2,  3,  4,  0,  1,  2,  3,
        4, 11, 12, 13, 14, 15, 11, 12, 13, 14, 15, 11, 12, 13, 14, 15, 11,
       12, 13, 14, 15, 11, 12, 13, 14, 15, 13, 14, 15, 16, 13, 14, 15, 16,
       13, 14, 15, 16, 13, 14, 15, 16,  0, 16, 17, 16, 17,  0,  1,  2,  3,
        0,  1,  2,  3,  0,  1,  2,  3,  0,  1,  2,  3, 21, 22, 21, 22,  0,
        1,  2,  3,  4,  0,  1,  2,  3,  4,  0,  1,  2,  3,  4,  0,  1,  2,
        3,  4,  0,  1,  2,  3,  4, 26, 27, 28, 29, 30, 26, 27, 28, 29, 30,
       26, 27, 28, 29, 30, 26, 27, 28, 29, 30, 26, 27, 28, 29, 30,  0, 30,
       31, 32, 33, 34, 30, 31, 32, 33, 34, 30, 31, 32, 33, 34, 30, 31, 32,
       33, 34, 30, 31, 32, 33, 34,  0,  1,  2,  3,  4,  0,  1,  2,  3,  4,
        0,  1,  2,  3,  4,  0,  1,  2,  3,  4,  0,  1,  2,  3,  4, 31, 32,
       33, 34, 35, 31, 32